# Containers

#### Docker implements the Open Container Initiative (OCI) specifications. This means everything you learn in this chapter applies to other container runtimes and platforms that implement the OCI specifications.

#### A container is the runtime instance of an image. In the same way that you can start a virtual machine (VM) from a virtual machine template, you start one or more containers from a single image. The big difference between a VM and a container is that containers are smaller, faster, and more portable.

#### The simplest way to start a container is with the docker run command. The command can take a lot of arguments, but in its most basic form you tell it an image to use and an app to run: docker run <image> <app>. The following command will start a new container based on the Ubuntu Linux image and start a Bash shell.

- $ docker run -it ubuntu /bin/bash` 
- The -it flags connect your current terminal window to the container’s shell.
- You can manually stop a running container with docker stop and restart it with docker start. To get rid of a container forever, you have to explicitly delete it with docker rm.


## Containers vs VMs

#### Containers and VMs both need a host to run on. The host can be anything from your laptop, a bare metal server in your data center, or an instance in the public cloud.
#### Let’s assume a requirement where your business has a single physical server that needs to run 4 business applications.
#### In the VM model, the physical server is powered on and the hypervisor boots. Once booted, the hypervisor claims all physical resources such as CPU, RAM, storage, and network cards. It then carves these hardware resources into virtual constructs that look smell and feel exactly like the real thing. It then packages them into a software construct called a virtual machine (VM). We take those VMs and install an operating system and application on each one.

## Things are a bit different in the container model.

#### The server is powered on and the OS boots. In this container model the host’s OS claims all hardware resources. Next you install a container engine such as Docker. The container engine then carves-up the OS resources (process tree, filesystem, network stack etc) and packages them into virtual operating systems called containers. Each container looks smells and feels just like a real OS. Inside of each container we run an application.

#### If we assume the same scenario of a single physical server needing to run 4 business applications, we’d carve the OS into 4 containers and run a single application inside each. 

#### At a high level, hypervisors perform hardware virtualization — they carve up physical hardware resources into virtual versions called VMs. On the other hand, containers perform OS virtualization — they carve OS resources into virtual versions called containers.

#### Another thing to consider is application start times. Containers start a lot faster than VMs because they only have to start the application — the kernel is already up and running on the host. In the VM model, each VM needs to boot a full OS before it can start the app.

#### This all makes the container model leaner and more efficient than the VM model. You can pack more applications onto less resources, start them faster, and pay less in licensing and admin costs, as well as present less of an attack surface!



## This command starts a simple container that will run a containerized version of Ubuntu Linux.
- docker run -it ubuntu:latest /bin/bash

#### docker run tells Docker to run a new container. The -it flags make the container interactive and attach it to your terminal. The ubuntu:latest argument tells Docker which image to use to start the container. Finally, /bin/bash is the application to run in the container.

#### When you hit Return, the Docker client packages up the command and POSTs it to the API server running on the Docker daemon. The Docker daemon accepts the command and searches the host’s local image repository to see if it already has a copy of the image. In our example it didn’t, so it went to Docker Hub to see if it could find it there. It found it, pulled it locally, and stored it in the local cache.

#### Typing exit while logged in to the container will terminate the Bash process and the whole container will exit (terminate). This is because a container cannot exist without its designated main process. This is true of Linux and Windows containers — killing the main process in the container will kill the container.